In [1]:
!pip install pandas numpy matplotlib prophet streamlit plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

np.random.seed(42)

dates = pd.date_range(start="2020-01-01", end="2024-12-31", freq="W")
products = ["Toothpaste", "Toothbrush", "Mouthwash"]

data = []

for p in products:
    base = np.random.randint(800, 1500)
    trend = np.linspace(0, 300, len(dates))
    seasonality = 200 * np.sin(np.linspace(0, 12 * np.pi, len(dates)))
    noise = np.random.normal(0, 80, len(dates))
    promo_flag = np.random.choice([0, 1], len(dates), p=[0.8, 0.2])

    sales = base + trend + seasonality + noise
    sales += promo_flag * np.random.randint(200, 500)

    for d, s, promo in zip(dates, sales, promo_flag):
        data.append([d, p, max(0, round(s)), promo])

df = pd.DataFrame(data, columns=["date", "product", "weekly_sales", "promotion"])
df.to_csv("colgate_sales_data.csv", index=False)

df.head()

,date,product,weekly_sales,promotion
0,2020-01-05,Toothpaste,858,0
1,2020-01-12,Toothpaste,973,0
2,2020-01-19,Toothpaste,999,0
3,2020-01-26,Toothpaste,1099,0
4,2020-02-02,Toothpaste,943,0


In [3]:
from prophet import Prophet

models = {}

for product in products:
    temp = df[df["product"] == product].copy()
    temp = temp.rename(columns={"date": "ds", "weekly_sales": "y"})

    m = Prophet(seasonality_mode='multiplicative')
    m.fit(temp)
    models[product] = m

import joblib
for p in models:
    joblib.dump(models[p], f"{p}_prophet_model.pkl")

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [4]:
forecast_output = {}

for p in products:
    future = models[p].make_future_dataframe(periods=20, freq='W')
    forecast = models[p].predict(future)
    forecast_output[p] = forecast
